In [1]:
# Step 1: Install required libraries
!pip install sentence-transformers pymupdf pymilvus

# Step 2: Extract text from the PDF
import fitz  # PyMuPDF

pdf_path = "sample.pdf"

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    return text

extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text[:1000])  # Print the first 1000 characters of the extracted text

# Step 3: Generate embeddings using PyTorch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Split text into sections
text_sections = extracted_text.split(". ")  # Simple sentence split by period

def generate_embeddings(text_sections):
    embeddings = model.encode(text_sections, convert_to_tensor=True)
    return embeddings

embeddings = generate_embeddings(text_sections)

print(f"Generated {len(embeddings)} embeddings.")
print(embeddings[:2])  # Show embeddings for the first two sections

# Step 4: Store embeddings in Milvus
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection

# Connect to Milvus
connections.connect("default", host="localhost", port="19530")

# Define the schema for the collection
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),  # Adjust dimensions to match your model
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2000),  # Store corresponding text
]

schema = CollectionSchema(fields, description="Document section embeddings")
collection = Collection(name="document_embeddings", schema=schema)

# Insert embeddings and text into Milvus
ids = list(range(len(embeddings)))
text_data = [section[:2000] for section in text_sections]  # Truncate text to fit VARCHAR limit
collection.insert([ids, embeddings.cpu().numpy(), text_data])

# Load the collection into memory
collection.load()

print("Embeddings successfully stored in Milvus!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 14.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 18.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 38.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.8.0
    Uninstalling setuptools-67.8.0:
      Successfully uninstalled setuptools-67.8.0


FileNotFoundError: no such file: 'sample.pdf'